In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import os

In [ ]:
def LoadAudio(Crema):
    emotions = []
    timeLines=[]
    Audio_List=[]
    samp_freq=[]

    for wav in os.listdir(Crema):
        path=Crema+"\\"+wav
        audio,sampling_freq=lr.load(path,duration=2.5, offset=0.6)
        Audio_List.append(audio)
        samp_freq.append(sampling_freq)
        timeLines.append(np.arange(0,len(audio))/sampling_freq)
        info = wav.partition(".wav")[0].split("_")
        if info[2] == 'SAD':
            emotions.append(0)
        elif info[2] == 'ANG':
            emotions.append(1)
        elif info[2] == 'DIS':
            emotions.append(2)
        elif info[2] == 'FEA':
            emotions.append(3)
        elif info[2] == 'HAP':
            emotions.append(4)
        elif info[2] == 'NEU':
            emotions.append(5)
        else:
            emotions.append(6)
            
    return Audio_List,timeLines,samp_freq,emotions

In [ ]:
Audio_List,timeLines,samp_freq,Labels= LoadAudio("C:\\Users\\fruty\\Downloads\\AudioWAV")
print(len(Audio_List))
print(len(timeLines))
print(len(Labels))

In [ ]:
#play first ten audio and plot them 
import sounddevice as sd
for j in range(0,6):
    fig,ax=plt.subplots()
    ax.plot(timeLines[j],Audio_List[j])
    ax.set(xlabel='Time',ylabel='sound amplitude')
    plt.show()
    sd.play(Audio_List[j], samp_freq[j])
    status = sd.wait()  # Wait until file is done playing

In [ ]:
print(lr.get_duration(Audio_List[0]))

In [ ]:
# Zero Crossing Rate
def zcr(data, frame_length=2048, hop_length=512):
    zcr = lr.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)

    return np.squeeze(zcr)

In [ ]:
def energy(data, frame_length=2048, hop_length=512):
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])

    return en / frame_length

In [ ]:
def extract_features(Audio_List, samp_freq, frame_length=2048, hop_length=512):
    result = np.array([])
    result = np.hstack((result,
                        zcr(Audio_List, frame_length, hop_length),
                        np.mean(energy(Audio_List, frame_length, hop_length),axis=0),
                        # np.mean(entropy_of_energy(data, frame_length, hop_length), axis=0),
                        # rmse(data, frame_length, hop_length),
                        # spc(data, sr, frame_length, hop_length),
                        # spc_entropy(data, sr),
                        # spc_flux(data),
                        # spc_rollof(data, sr, frame_length, hop_length),
                        # chroma_stft(data, sr, frame_length, hop_length),
                        # mel_spc(data, sr, frame_length, hop_length, flatten=True)
                        # mfcc(data, sr, frame_length, hop_length)
                                    ))
    return result

In [ ]:
features=[]
for i in range(len(Audio_List)):
    features.append(extract_features(Audio_List[i],samp_freq[i]))

In [ ]:
extracted_df = pd.DataFrame(features)
extracted_df["labels"] = Labels
extracted_df = extracted_df.fillna(0)

In [ ]:

def mel_spc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    mel = lr.feature.melspectrogram(y=data, sr=sr)
    return np.squeeze(mel.T) if not flatten else np.ravel(mel.T)

In [ ]:
def extract_features2(data, sr, frame_length=2048, hop_length=512):
    result = np.array([])
    result = np.hstack((result,mel_spc(data, sr, frame_length, hop_length,flatten = True)))
    print()
    return result

In [ ]:
features2=[]
for i in range(len(Audio_List)):
    features2.append(extract_features2(Audio_List[i],samp_freq[i]))

In [ ]:
extracted_df2 = pd.DataFrame(features2)
extracted_df2["labels"] = Labels
extracted_df2 = extracted_df2.fillna(0)

In [ ]:
print(extracted_df2.head())

In [ ]:
def train_test_split(data):
    newdf1 = np.random.rand(len(data)) < 0.7
    train = data[newdf1]
    test = data[~newdf1]
    return train,test

In [ ]:

train_data=[]
test_data=[]
validation_data=[]
train_data,test_data=train_test_split(extracted_df)

In [ ]:
train_data2=[]
test_data2=[]
validation_data2=[]
train_data2,test_data2=train_test_split(extracted_df2)

In [ ]:

print(len(train_data))
print(len(test_data))
print(len(train_data2))
print(len(test_data2))

In [ ]:
def validation_splitting(trainSet):
  validation_data=[]
  trainWithout_val=[]
  output = np.random.rand(len(trainSet)) < 0.95
  trainWithout_val = trainSet[output]
  validation_data= trainSet[~output]
  return validation_data,trainWithout_val

In [ ]:
validation_data1D=[]
trainWithout_val1D=[]
validation_data2D=[]
trainWithout_val2D=[]
validation_data1D,trainWithout_val1D=validation_splitting(train_data)
validation_data2D,trainWithout_val2D=validation_splitting(train_data2)